## Imports


In [1]:
from numpy import pi

import qutip as qt

from rich import print as pprint

from oqd_compiler_infrastructure import PrettyPrint, Post, FixedPoint, Chain, In, Pre

from oqd_core.compiler.math.passes import simplify_math_expr, evaluate_math_expr


from oqd_core.compiler.math.rules import (
    DistributeMathExpr,
    ProperOrderMathExpr,
    PartitionMathExpr,
    PrintMathExpr,
)


from trical.light_matter2 import *
from trical.backend2.qutip.codegen import QutipCodeGeneration
from trical.backend2.qutip.vm import QutipVM

## Atomic Circuit


In [2]:
downstate = Level(
    principal=6,
    spin=1 / 2,
    orbital=0,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=0,
    spin_orbital_nuclear_magnetization=0,
    energy=0,
)
upstate = Level(
    principal=6,
    spin=1 / 2,
    orbital=0,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=1,
    spin_orbital_nuclear_magnetization=0,
    energy=2 * pi * 12.643e9,
)
estate = Level(
    principal=5,
    spin=1 / 2,
    orbital=1,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=1,
    spin_orbital_nuclear_magnetization=-1,
    energy=2 * pi * 811.52e12,
)
estate2 = Level(
    principal=5,
    spin=1 / 2,
    orbital=1,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=1,
    spin_orbital_nuclear_magnetization=1,
    energy=2 * pi * 811.52e12,
)

transitions = [
    Transition(
        level1=downstate,
        level2=estate,
        einsteinA=1.23e8,
        multipole="E1",
    ),
    Transition(
        level1=downstate,
        level2=estate2,
        einsteinA=1.23e8,
        multipole="E1",
    ),
    Transition(
        level1=upstate,
        level2=estate,
        einsteinA=1.23e8,
        multipole="E1",
    ),
    Transition(
        level1=upstate,
        level2=estate2,
        einsteinA=1.23e8,
        multipole="E1",
    ),
]

Yb171 = Ion(
    mass=171,
    charge=1,
    position=[0, 0, 0],
    levels=[downstate, upstate, estate, estate2],
    transitions=transitions,
)

COM_x = Phonon(energy=2 * pi * 5e6, eigenvector=[1, 0, 0])
COM_y = Phonon(energy=2 * pi * 5e6, eigenvector=[0, 1, 0])
COM_z = Phonon(energy=2 * pi * 1e6, eigenvector=[0, 0, 1])

system = System(
    ions=[
        Yb171,
    ],
    modes=[
        COM_x,
        COM_y,
        COM_z,
    ],
)

beam = Beam(
    transition=transitions[0],
    rabi=2 * pi * 1e9,
    detuning=2 * pi * 1e6,
    phase=0,
    polarization=[1, 1, 0],
    wavevector=[0, 2 * pi / 0.02371213, 0],
    target=0,
)

beam2 = Beam(
    transition=transitions[2],
    rabi=2 * pi * 1e9,
    detuning=2 * pi * 1e6,
    phase=0,
    polarization=[1, 1, 0],
    wavevector=[0, 2 * pi / 0.02371213, 0],
    target=0,
)

protocol = SequentialProtocol(
    sequence=[
        ParallelProtocol(
            sequence=[
                Pulse(beam=beam, duration=1e-9),
                Pulse(beam=beam2, duration=1e-9),
            ]
        ),
        Pulse(beam=beam, duration=1e-9),
    ]
)

circuit = AtomicCircuit(system=system, protocol=protocol)

## Construct Hamiltonian


In [3]:
compiler = Post(ConstructHamiltonian())

op = compiler(circuit)

printer = Post(PrettyPrint())
pprint(printer(op))

AtomicEmulatorCircuit
  - base: OperatorAdd
    - op1: OperatorAdd
      - op1: OperatorAdd
        - op1: OperatorAdd
          - op1: Zero
          - op2: OperatorKron
            - op1: OperatorKron
              - op1: OperatorKron
                - op1: OperatorAdd
                  - op1: OperatorAdd
                    - op1: OperatorAdd
                      - op1: OperatorAdd
                        - op1: Zero
                        - op2: OperatorScalarMul
                          - op: KetBra
                            - ket: int(0)
                            - bra: int(0)
                          - coeff: WaveCoefficient
                            - amplitude: MathNum
                              - value: float(0.0)
                            - frequency: MathNum
                              - value: int(0)
                            - phase: MathNum
                              - value: int(0)
                      - op2: OperatorScalarMul
                        - op: KetBra
                          - ket: int(1)
                          - bra: int(1)
                        - coeff: WaveCoefficient
                          - amplitude: MathNum
                            - value: float(79438311838.67151)
                          - frequency: MathNum
                            - value: int(0)
                          - phase: MathNum
                            - value: int(0)
                    - op2: OperatorScalarMul
                      - op: KetBra
                        - ket: int(2)
                        - bra: int(2)
                      - coeff: WaveCoefficient
                        - amplitude: MathNum
                          - value: float(5098930540482378.0)
                        - frequency: MathNum
                          - value: int(0)
                        - phase: MathNum
                          - value: int(0)
                  - op2: OperatorScalarMul
                    - op: KetBra
                      - ket: int(3)
                      - bra: int(3)
                    - coeff: WaveCoefficient
                      - amplitude: MathNum
                        - value: float(5098930540482378.0)
                      - frequency: MathNum
                        - value: int(0)
                      - phase: MathNum
                        - value: int(0)
                - op2: Identity
              - op2: Identity
            - op2: Identity
        - op2: OperatorKron
          - op1: OperatorKron
            - op1: OperatorKron
              - op1: Identity
              - op2: OperatorScalarMul
                - op: OperatorMul
                  - op1: Creation
                  - op2: Annihilation
                - coeff: WaveCoefficient
                  - amplitude: MathNum
                    - value: float(31415926.535897933)
                  - frequency: MathNum
                    - value: int(0)
                  - phase: MathNum
                    - value: int(0)
            - op2: Identity
          - op2: Identity
      - op2: OperatorKron
        - op1: OperatorKron
          - op1: OperatorKron
            - op1: Identity
            - op2: Identity
          - op2: OperatorScalarMul
            - op: OperatorMul
              - op1: Creation
              - op2: Annihilation
            - coeff: WaveCoefficient
              - amplitude: MathNum
                - value: float(31415926.535897933)
              - frequency: MathNum
                - value: int(0)
              - phase: MathNum
                - value: int(0)
        - op2: Identity
    - op2: OperatorKron
      - op1: OperatorKron
        - op1: OperatorKron
          - op1: Identity
          - op2: Identity
        - op2: Identity
      - op2: OperatorScalarMul
        - op: OperatorMul
          - op1: Creation
          - op2: Annihilation
        - coeff: WaveCoefficient
          - amplitude: MathNum
            - value: float(6283185.307179586)
          - freque

### Simplification


In [4]:
from oqd_compiler_infrastructure import RewriteRule
from oqd_core.interface.math import MathNum


class Prune(RewriteRule):
    def map_OperatorAdd(self, model):
        if isinstance(model.op1, Zero):
            return model.op2
        if isinstance(model.op2, Zero):
            return model.op1

    def map_OperatorMul(self, model):
        if isinstance(model.op1, Zero) or isinstance(model.op2, Zero):
            return Zero()

    def map_OperatorScalarMul(self, model):
        if isinstance(
            model.coeff, WaveCoefficient
        ) and model.coeff.amplitude == MathNum(value=0):
            return Zero()

    def map_OperatorKron(self, model):
        if isinstance(model.op1, Zero) or isinstance(model.op2, Zero):
            return Zero()

    def map_CoefficientAdd(self, model):
        if isinstance(
            model.coeff1, WaveCoefficient
        ) and model.coeff1.amplitude == MathNum(value=0):
            return model.coeff2
        if isinstance(
            model.coeff2, WaveCoefficient
        ) and model.coeff2.amplitude == MathNum(value=0):
            return model.coeff1

    def map_Wave(self, model):
        if isinstance(
            model.lamb_dicke, WaveCoefficient
        ) and model.lamb_dicke.amplitude == MathNum(value=0):
            return Identity()


class PruneZeroPowers(RewriteRule):
    def map_MathPow(self, model):
        if model.expr1 == MathNum(value=0):
            return MathNum(value=0)


simplify = Chain(
    Chain(
        FixedPoint(Post(DistributeMathExpr())),
        FixedPoint(Post(ProperOrderMathExpr())),
        FixedPoint(Post(PartitionMathExpr())),
        FixedPoint(Post(PruneZeroPowers())),
    ),
    simplify_math_expr,
    FixedPoint(Post(Prune())),
    Chain(
        FixedPoint(Post(DistributeMathExpr())),
        FixedPoint(Post(ProperOrderMathExpr())),
        FixedPoint(Post(PartitionMathExpr())),
        FixedPoint(Post(PruneZeroPowers())),
    ),
    simplify_math_expr,
)

## Qutip Intepreter


In [5]:
def qutip_compiler(circuit, fock_cutoff=1):
    analysis = In(AnalyseHilbertSpace())

    analysis(circuit)

    hilbert_space = analysis.children[0].hilbert_space

    compiler = Chain(
        Post(ConstructHamiltonian()),
        simplify,
        Post(QutipCodeGeneration(fock_cutoff=fock_cutoff)),
    )

    compiled_circuit = compiler(circuit)

    return compiled_circuit, hilbert_space, fock_cutoff


def qutip_intepreter(compiled_circuit, hilbert_space, fock_cutoff, timestep):
    vm = Pre(
        QutipVM(
            hilbert_space=hilbert_space,
            fock_cutoff=fock_cutoff,
            timestep=timestep,
        )
    )

    vm(compiled_circuit)

    return vm.children[0].result

In [6]:
compiled_circuit, hilbert_space, fock_cutoff = qutip_compiler(circuit, 10)

printer = Post(PrettyPrint())
pprint(printer(compiled_circuit))

QutipExperiment
  - base: function(<function QutipCodeGeneration.map_OperatorAdd.<locals>.<lambda> at 0x00000222370AB250>)
  - sequence: list
    - 0: QutipGate
      - hamiltonian: function(<function QutipCodeGeneration.map_OperatorAdd.<locals>.<lambda> at 
0x00000222375C9120>)
      - duration: float(1e-09)
    - 1: QutipGate
      - hamiltonian: function(<function QutipCodeGeneration.map_OperatorKron.<locals>.<lambda> at 
0x00000222375CA5F0>)
      - duration: float(1e-09)

In [ ]:
result = qutip_intepreter(
    compiled_circuit,
    hilbert_space=[10, "f", "f", "f"],
    fock_cutoff=fock_cutoff,
    timestep=1e-11,
)

In [ ]:
# from matplotlib import pyplot as plt

# k = slice(None, None, 5)

# plt.plot(
#     result["tspan"][k],
#     [s.ptrace(0).diag()[:4] for s in result["states"][k]],
# )